In [1]:
%load_ext autoreload
%autoreload

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig,make_sensitivity_fig

In [2]:
comp = Component.demo()
fm = FailureMode.demo()

In [3]:
comp.__dict__["fm"]["early_life"].__dict__

{'_name': 'early_life',
 'units': 'years',
 'up_to_date': True,
 'dists': {'untreated': Distribution(name=untreated, alpha=1000000.0, beta=0.5, gamma=0.0,
  'init': Distribution(name=init, alpha=1000000.0, beta=0.5, gamma=0.0},
 'indicators': {'instant': <pof.indicator.ConditionIndicator at 0x1f8bb2a9a60>,
  'slow_degrading': <pof.indicator.ConditionIndicator at 0x1f8bb2a9880>,
  'fast_degrading': <pof.indicator.ConditionIndicator at 0x1f8bb2a9370>},
 'conditions': {'instant': {'name': 'instant',
   'perfect': 1,
   'failed': 0,
   'pf_curve': 'step',
   'pf_interval': 1,
   'pf_std': 0}},
 'consequences': {},
 'tasks': {'inspection': <pof.task.Inspection at 0x1f8bb2bf220>,
  'on_condition_replacement': <pof.task.ConditionTask at 0x1f8bb2bf160>,
  'on_failure_replacement': <pof.task.ConditionTask at 0x1f8bb2bf6d0>},
 'init_states': {'initiation': False, 'detection': False, 'failure': False},
 'states': {'initiation': False, 'detection': False, 'failure': False},
 'active': True,
 '_pf_

In [4]:
comp.__dict__["fm"]["fast_aging"].__dict__['active'] = False

In [5]:
# Turn off logging
logging.getLogger().setLevel(logging.INFO)
#config['Load']['on_error_use_default'] ='False'

filename = os.path.dirname(os.getcwd()) + r"\data\inputs\Asset Model - Demo.xlsx"

aml = AssetModelLoader(filename)
comp_data = aml.load()
comp = Component.from_dict(comp_data['pole'])

In [6]:
comp.__dict__["fm"]

{'termites': <pof.failure_mode.FailureMode at 0x1f886e32ee0>,
 'fungal decay _ external': <pof.failure_mode.FailureMode at 0x1f89c8e3a00>,
 'fungal decay _ internal': <pof.failure_mode.FailureMode at 0x1f89c90a280>,
 'lightning': <pof.failure_mode.FailureMode at 0x1f89c90a370>,
 'fire_damage': <pof.failure_mode.FailureMode at 0x1f89c90aee0>,
 'impact': <pof.failure_mode.FailureMode at 0x1f89c914160>,
 'weathering': <pof.failure_mode.FailureMode at 0x1f89c914580>}

In [7]:
comp.__dict__["fm"]["fungal decay _ external"].__dict__['active'] = False
comp.__dict__["fm"]["fungal decay _ internal"].__dict__['active'] = False
comp.__dict__["fm"]["lightning"].__dict__['active'] = False
comp.__dict__["fm"]["fire_damage"].__dict__['active'] = False
comp.__dict__["fm"]["impact"].__dict__['active'] = False
comp.__dict__["fm"]["weathering"].__dict__['active'] = False

In [8]:
comp.__dict__["fm"]["termites"].__dict__["tasks"]["CAT3/4 replacement"].__dict__

{'_name': 'CAT3/4 replacement',
 'units': 'years',
 'up_to_date': True,
 'task_type': 'ConditionTask',
 'task_category': 'Graphing purposes',
 'task_group_name': 'unknown',
 'trigger': 'condition',
 'active': True,
 '_package': NotImplemented,
 '_impacts_parent': NotImplemented,
 '_impacts_children': False,
 'cost': 3500.0,
 'labour': NotImplemented,
 'spares': NotImplemented,
 'equipment': NotImplemented,
 'consequence': <pof.consequence.Consequence at 0x1f89c8e3940>,
 'p_effective': 1.0,
 'triggers': {'state': {'initiation': True,
   'detection': True,
   'failure': False},
  'condition': {'wall_thickness': {'lower': 0.0, 'upper': 50},
   'external_diameter': {'lower': 0.0, 'upper': 50},
   'safety_factor': {'lower': 0.0, 'upper': 3}},
  'time': {}},
 'impacts': {'state': {'initiation': False,
   'detection': False,
   'failure': False},
  'condition': {'all': {'target': 1.0,
    'method': 'reduction_factor',
    'axis': 'condition'}},
  'system': ['component'],
  'time': {}},
 'stat

In [9]:
comp.mc_timeline(1000)
update_cost_fig(comp)

100%|██████████| 100/100 [00:00<00:00, 107.88it/s]


In [10]:
comp.mc_timeline(1000)
update_condition_fig(comp)

100%|██████████| 100/100 [00:00<00:00, 132.80it/s]


In [11]:
df = comp.get_update_ids()
df[5]

'pole-fm-termites-dists-untreated-alpha'

In [12]:
dropdown_list = []
for value in comp.get_update_ids():
    dropdown_dict = {}
    dropdown_dict["label"] = value
    dropdown_dict["value"] = value
    dropdown_list.append(dropdown_dict)


In [13]:
make_sensitivity_fig(comp, var_name="pole-fm-termites-dists-untreated-alpha", t_min=0, t_max=50, step_size=10, n_iterations=10)

100%|██████████| 10/10 [00:00<00:00, 188.67it/s]


In [14]:
comp.mc_timeline(10000)
update_pof_fig(comp)

100%|██████████| 100/100 [00:02<00:00, 37.31it/s]
